In [1]:
from data import RooftopDataset
from transforms import ToTensor
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import optim

In [2]:
data_root_path = 'images'
dataset = RooftopDataset(root=data_root_path, grayscale=True)
dataset.save_color_encoded_images()
import numpy as np
print(np.where(dataset[0][1]['masks'] == 1))
print(dataset[0][1])

filename: 121.png
nr labels: 5
filename: 241.png
nr labels: 3
filename: 270.png
nr labels: 37


IndexError: list index out of range

In [57]:
import os
from PIL import Image
from scipy import ndimage
mask_path = os.path.join(data_root_path, 'labels', dataset.masks[0])
print(mask_path)
mask_file_name = os.path.basename(mask_path)
color_codes = [10, 40, 70, 100, 130, 160, 190, 220, 250]
with Image.open(mask_path) as mask:
    mask = mask.convert('L')
mask_array = np.array(mask)
print(mask_array)
# make the picture binary. 100 is arbitrary but seems to work for dataset
mask_array[np.where(mask_array > 100)] = 255
mask_array[np.where(mask_array <= 100)] = 0
label_img, nr_labels = ndimage.label(mask_array)

for label in range(nr_labels):
    colored = np.array
    label_img[np.where(label_img == label + 1)] = color_codes[label]

images/labels/121.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [50]:
img = Image.fromarray(label_img, 'L')
img.save('my.png')
img.show()

In [64]:
colored = mask_array
colored[np.where(label_img == 5)] = 50

mask_n = Image.fromarray(colored, 'L')
mask_n.show()

In [21]:
dataset = RooftopDataset(root=data_root_path, grayscale=True, transforms=ToTensor())

In [4]:
# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, min_size=256)

# replace the classifier with a new one
num_classes = 2  # 1 class (rooftop) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, min_size=128)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [12]:
def train_one_epoch(model, optimizer, data_loader):
    model.train()

    for i, (images, targets) in enumerate(data_loader):
        targets = list(target for target in targets)
        
        loss_dict = model(images, targets)
        print(loss_dict)

        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

In [7]:
# get the model using our helper function
model = get_model_instance_segmentation(num_classes)
# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader)
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

NameError: name 'optimizer' is not defined

In [17]:
from engine import evaluate
import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    device = torch.device('cpu')
    # our dataset has two classes only - background and person
    num_classes = 2
    
    to_grayscale = True

    dataset = RooftopDataset(root=data_root_path,
                             transforms=ToTensor(),
                             #grayscale=to_grayscale
                            )
    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset_test = RooftopDataset(root=data_root_path, transforms=ToTensor())

    dataset = torch.utils.data.Subset(dataset, indices[:-2])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-2:])

    data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=2, shuffle=True, num_workers=1,
            collate_fn=utils.collate_fn
    )

    data_loader_test = torch.utils.data.DataLoader(
            dataset_test, batch_size=1, shuffle=False, num_workers=1,
            collate_fn=utils.collate_fn
    )

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    # model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader)
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [18]:
%tb
main()

NameError: name 'net' is not defined

{'loss_classifier': tensor(0.7139, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.1773, grad_fn=<DivBackward0>), 'loss_mask': tensor(4.1791, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.4120, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0314, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(0.9929, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(4.8531, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(3.5592, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(inf, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'lo

AssertionError: 
The NVIDIA driver on your system is too old (found version 9010).
Please update your GPU driver by downloading and installing a new
version from the URL: http://www.nvidia.com/Download/index.aspx
Alternatively, go to: https://pytorch.org to install
a PyTorch version that has been compiled with your version
of the CUDA driver.